# 优化模型参数

现在我们有了模型和数据，是时候通过优化数据参数来训练、验证和测试我们的模型了。训练模型是一个迭代的过程；在每一次迭代中，模型都会对输出进行猜测，计算猜测的误差（损失），收集误差相对于其参数的导数（正如我们在上一节中所看到的），并使用梯度下降法优化这些参数。如需了解这一过程的更多细节，请观看 3Blue1Brown 提供的[反向传播视频](https://www.youtube.com/watch?v=tIeHLnjs5U8)。

## 依赖代码

我们从前面的[数据集和数据加载器](../DatasetsDataLoaders/datasets.ipynb)以及[构建模型](../BuildModel/build_model.ipynb)部分加载代码。

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 26421880/26421880 [00:03<00:00, 7618927.33it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 213701.05it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3455669.04it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 22657163.69it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## 超参数

超参数是可调参数，可让您控制模型优化过程。不同的超参数值会影响模型训练和收敛速度（了解有关超参数调整的[更多信息](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)）。 

我们为训练定义了以下超参数：
+ 时序数（Number of Epochs）- 在数据集上迭代的次数；
+ 批量大小（Batch Size）- 在更新参数之前通过网络传播的数据样本数量；
+ 学习率（Learning Rate）- 在每个批次/时序更新模型参数的数量。数值越小，学习速度越慢，而数值过大则可能导致训练过程中出现不可预测的行为；

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 优化循环

设置好超参数后，我们就可以通过优化循环来训练和优化模型。优化循环的每次迭代称为一个 Epoch。

每个 Epoch 由两个主要部分组成：

+ 训练循环 -- 遍历训练数据集，尝试收敛到最佳参数。
+ 验证/测试循环--遍历测试数据集，检查模型性能是否在提高。

让我们简单熟悉一下训练循环中使用的一些概念。跳转至优化循环的[完整实现](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html#full-impl-label)。

### 损失函数

当遇到一些训练数据时，我们未经训练的网络很可能无法给出正确答案。损失函数衡量的是获得的结果与目标值的不相似程度，我们希望在训练过程中将损失函数最小化。为了计算损失，我们使用给定数据样本的输入进行预测，并与真实的数据标签值进行比较。

常见的损失函数包括用于回归任务的 `nn.MSELoss`（均方误差）和用于分类的 `nn.NLLLoss`（负对数似然）。`nn.CrossEntropyLoss` 结合了 `nn.LogSoftmax` 和 `nn.NLLLoss`。

我们将模型的输出 logits 传递给 `nn.CrossEntropyLoss`，它将对 logits 进行**归一化**，并计算预测误差。

In [3]:
# 初始化损失函数
loss_fn = nn.CrossEntropyLoss()

### 优化器

优化是在每个训练步骤中调整模型参数以减少模型误差的过程。优化算法定义了如何执行这一过程（在本例中，我们使用**随机梯度下降算法**）。所有优化逻辑都封装在优化器对象中。在这里，我们使用 SGD 优化器；此外，PyTorch 中还有许多不同的优化器，如 `ADAM` 和 `RMSProp`，它们对不同类型的模型和数据效果更好。

我们通过注册需要训练的模型参数，并传递学习率、超参数来**初始化优化器**。

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环中，优化分为三个步骤：

+ 调用 `optimizer.zero_grad()` 重置模型参数的梯度。梯度默认是累加的；为了防止重复计算，我们在每次迭代时都明确地将梯度清零。
+ 调用 `loss.backward()`，反向传播预测损失。PyTorch 将损失与每个参数相关的梯度沉淀下来。
+ 一旦有了梯度，我们就调用 `optimizer.step()`，根据后向传递中收集的梯度来调整参数。

## 完整实现

我们定义了 `train_loop`（循环优化代码）和 `test_loop`（根据测试数据评估模型性能）。

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # 将模型设置为训练模式，这对批量归一化和丢弃层很重要 
    # 在这种情况下没有必要，但为最佳实践添加了这一功能
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 计算预测值和损失值
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 将模型设置为评估模式，对批量归一化和 dropout 很重要 
    # 在这种情况下没有必要，但为最佳实践而添加
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # 使用 torch.no_grad() 对模型进行评估，可确保在测试模式下不计算梯度 
    # 还可减少不必要的梯度计算和内存使用，适用于 requires_grad=True 的张量因子
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

我们将初始化损失函数和优化器，并将其传递给 `train_loop` 和 `test_loop`。请随意增加 `epoch` 次数，以跟踪模型性能的提高。

In [6]:
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------------------
loss: 2.307146  [   64/60000]


[ROCT-Thunk-Interface/src/hymgr.c:309, WARNING]: Version mismatch


loss: 2.289888  [ 6464/60000]
loss: 2.269248  [12864/60000]
loss: 2.259783  [19264/60000]
loss: 2.245266  [25664/60000]
loss: 2.219535  [32064/60000]
loss: 2.226249  [38464/60000]
loss: 2.193237  [44864/60000]
loss: 2.183918  [51264/60000]
loss: 2.150151  [57664/60000]
Test Error: 
 Accuracy: 41.1%, Avg loss: 2.146558 

Epoch 2
--------------------------------
loss: 2.162424  [   64/60000]
loss: 2.147504  [ 6464/60000]
loss: 2.087000  [12864/60000]
loss: 2.104644  [19264/60000]
loss: 2.044930  [25664/60000]
loss: 1.993416  [32064/60000]
loss: 2.019670  [38464/60000]
loss: 1.936099  [44864/60000]
loss: 1.940492  [51264/60000]
loss: 1.864422  [57664/60000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.862761 

Epoch 3
--------------------------------
loss: 1.901004  [   64/60000]
loss: 1.867206  [ 6464/60000]
loss: 1.744931  [12864/60000]
loss: 1.790872  [19264/60000]
loss: 1.665342  [25664/60000]
loss: 1.631033  [32064/60000]
loss: 1.649889  [38464/60000]
loss: 1.542201  [44864/60000]
loss

## Further Reading

+ [损失函数](https://pytorch.org/docs/stable/nn.html#loss-functions)
+ [torch.optim](https://pytorch.org/docs/stable/optim.html)
+ [热启动训练模型](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)